In [1]:
# Details:
# Study: Brain Network Organization and Behavior
# Note: This notebook was used to convert dicoms to bids compliant nii files


# Credits:
# Built from this guide: http://nipy.org/heudiconv/#1
# Written by Richard Huskey


# Notes and dependencies
# Requres Docker: https://docs.docker.com/install/linux/docker-ce/ubuntu/#install-docker-ce-1
# When including fieldmap, note these instructions for echo times: https://github.com/rordenlab/dcm2niix/issues/139

In [2]:
%%bash
docker pull nipy/heudiconv:latest

# Pulls the latest version of heudiconv which converts your .dcm to .nii in a BIDS compliant data structure

latest: Pulling from nipy/heudiconv
Digest: sha256:2ab866fca794794ccb80bbdf631a7a7e9df8eae85e8ee98f3ec5e1453edc01b8
Status: Image is up to date for nipy/heudiconv:latest


In [3]:
%%bash
mkdir bids_nii
mkdir heudiconv_temp

# Make an output directory for your BIDS compliant dataset
# Also makes an output directory for the heudiconv info files (needed to make heuristic file, below)
# Makes both in the current working directory

In [4]:
# NOTE: sub count starts at 005 (subs 001-004 were pilot scans using a different procedure)
# NOTE: sub 010 excluded due to abnormal radiological reading
# NOTE: subs 027 and 038 exhibited contraindication to scanning and therefore are excluded


# Define a shell array that includes all your subjects
# Runs heudiconv without any conversion, just passing in DICOMs and getting information about them
# Do NOT save the output to the same directory as you plan on saving your final BIDS formatted dataset
# Be sure to update the data and output paths

In [5]:
%%bash
subs=( 005 006 007 008 009 011 012 013 014 015 016 017 018 019 020 021 022 023 024 025 026 028 029 030 031 032 033 034 035 036 037 039 040 041 042 )

for element in "${subs[@]}"; do docker run --rm -v /home/huskeyadmin/Desktop/main_study_bids:/data:ro -v /home/huskeyadmin/Desktop/main_study_bids/heudiconv_temp:/output nipy/heudiconv:latest -d /data/{subject}/* -s "$element" -f convertall -c none -o /output; done

INFO: Running heudiconv version 0.5.2-dev
INFO: Need to process 1 study sessions
INFO: PROCESSING STARTS: {'session': None, 'outdir': '/output/', 'subject': '005'}
INFO: Processing 1378 dicoms
INFO: Analyzing 1378 dicoms
INFO: Generated sequence info with 20 entries
INFO: PROCESSING DONE: {'session': None, 'outdir': '/output/', 'subject': '005'}
INFO: Running heudiconv version 0.5.2-dev
INFO: Need to process 1 study sessions
INFO: PROCESSING STARTS: {'session': None, 'outdir': '/output/', 'subject': '006'}
INFO: Processing 1618 dicoms
INFO: Analyzing 1618 dicoms
INFO: Generated sequence info with 20 entries
INFO: PROCESSING DONE: {'session': None, 'outdir': '/output/', 'subject': '006'}
INFO: Running heudiconv version 0.5.2-dev
INFO: Need to process 1 study sessions
INFO: PROCESSING STARTS: {'session': None, 'outdir': '/output/', 'subject': '007'}
INFO: Processing 1618 dicoms
INFO: Analyzing 1618 dicoms
INFO: Generated sequence info with 20 entries
INFO: PROCESSING DONE: {'session': No

In [6]:
# Use the output of the dicominfo.tsv file to create heuristic keys for each of your scans
# The .tsv file is found in the current working directory under .heudiconv/{subject}/info/dicominfo.tsv
# Use specific info in the column to define features (below) that can only belong to a single run

In [7]:
%%writefile dcm2nii_heuristic.py

# Makes a heuristic file and then writes it to the current working directory
# NOTE: fmap is commented out due to open issue with hudiconv, see https://github.com/nipy/heudiconv/issues/249

import os

def create_key(template, outtype=('nii.gz',), annotation_classes=None):
    if template is None or not template:
        raise ValueError('Template must be valid format string')
    return template, outtype, annotation_classes

def infotodict(seqinfo):
    """Heuristic evaluator for determining which runs belong where
    
    allowed template fields - follow python string module:
    
    item: index within category
    subject: participant id
    seqitem: run number during scanning
    subindex: sub index within group
    """
     
    # Keys define the type of scan
    # Below extracts T1 and some task runs
    # Paths done in BIDS format
    
    t1w = create_key('sub-{subject}/anat/sub-{subject}_T1w')
    t2w = create_key('sub-{subject}/anat/sub-{subject}_T2w')
    # phasediff = create_key('sub-{subject}/fmap/sub-{subject}_phasediff')
    # magnitude = create_key('sub-{subject}/fmap/sub-{subject}_magnitude')
    game1 = create_key('sub-{subject}/func/sub-{subject}_task-game_run-01_bold')
    game2 = create_key('sub-{subject}/func/sub-{subject}_task-game_run-02_bold')
    game3 = create_key('sub-{subject}/func/sub-{subject}_task-game_run-03_bold')
    rest = create_key('sub-{subject}/func/sub-{subject}_task-rest_bold')
   
    #info = {t1w: [], t2w: [], magnitude: [], phasediff: [], game1: [], game2: [], game3: [], rest:[]}
    info = {t1w: [], t2w: [], game1: [], game2: [], game3: [], rest:[]}

    for idx, s in enumerate(seqinfo):
        # s is a named tuple with fields equal to the names of the columns
        # that are found in the dicominfo.tsv file
        if (s.dim3 == 176) and (s.dim4 == 1) and('MPRAGE' in s.protocol_name):
            info[t1w] = [s.series_id] # assign if a single scan meets criteria
        if (s.dim3 == 60) and (s.dim4 == 1) and('t2' in s.protocol_name):
            info[t2w] = [s.series_id] # assign if a single scan meets criteria
        #if (s.dim3 == 144) and (s.dim4 == 1) and('fieldmap' in s.protocol_name):
        #    info[magnitude] = [s.series_id] # assign if a single scan meets criteria
        #if (s.dim3 == 72) and (s.dim4 == 1) and('fieldmap' in s.protocol_name):
        #    info[phasediff] = [s.series_id] # assign if a single scan meets criteria
        if (s.dim3 == 72) and (s.dim4 == 185) and('run1' in s.protocol_name):
            info[game1] = [s.series_id] # assign if a single scan meets criteria
        if (s.dim3 == 72) and (s.dim4 == 185) and('run2' in s.protocol_name):
            info[game2] = [s.series_id] # assign if a single scan meets criteria
        if (s.dim3 == 72) and (s.dim4 == 185) and('run3' in s.protocol_name):
            info[game3] = [s.series_id] # assign if a single scan meets criteria
        if (s.dim3 == 72) and (s.dim4 == 310) and('resting' in s.protocol_name):
            info[rest] = [s.series_id] # assign if a single scan meets criteria
    return info

Writing dcm2nii_heuristic.py


In [8]:
# Converts dicoms to nii in a BIDS compliant format
# Use the same shell array you defined above
# Be sure to update the data and output paths

In [ ]:
%%bash
subs=( 005 006 007 008 009 011 012 013 014 015 016 017 018 019 020 021 022 023 024 025 026 028 029 030 031 032 033 034 035 036 037 039 040 041 042 )

for element in "${subs[@]}"; do docker run --rm -v /home/huskeyadmin/Desktop/main_study_bids:/data:ro -v /home/huskeyadmin/Desktop/main_study_bids/bids_nii:/output nipy/heudiconv:latest -d /data/{subject}/* -s "$element" -f /data/dcm2nii_heuristic.py -b -o /output; done

In [3]:
import webbrowser

# Check, is your data structure BIDS compliant?

url = 'https://' + 'bids-standard.github.io/bids-validator/'
webbrowser.open(url)

True